# Algorithm
Step 1. import URL handling modules - urllib.request,urllib.parse and urllib.error for opening, reading, parsing and displaying exceptions
Step 2. import BeautifulSoup - python package to scrape data from mentioned websites
Step 3. Provide the url of the CNN website as a string and store it in variable url
Step 4. Open the url using urllib.request.urlopen(url) function and store it in variable page
Step 5. Create a BeautifulSoup object named soup with the html content of the page
Step 6. Inspect the table class name("wsod_dataTable wsod_dataTableBigAlt ") containing the details of Most Active stocks only and store it in variable table_MostActive
Step 7. Use soup.find_all() method to detect the first table class with name table_MostActive and store it in variable tab
Step 8. Iterate each table row in the table class
Step 9. Iterate each table data in the table row 
Step 10. With each iteration of the table data, append table data to list l1
Step 11. With each iteration of the table row, append list l1 to list data_MostActive
Step 12. Slice the data_MostActive list from index 1 to the rest of the elements in the last , to remove the null element present in the first index 
Step 13. Iterate each element in data_MostActive list
Step 14. Retrieve only the ticker symbol from the elements in the zero th index and store it in variable tickersymbols
Step 15. Append the tickersymbols to list temp
Step 16. Provide the url of the Yahoo Finance website as a string and store it in variable ticketurl
Step 17. Iterate each element in temp list
Step 18. Request to get the website containing the information of each ticker symbol and store it in variable mystockhandle
Step 19. Create a BeautifulSoup object named soup with the html content of mystockhandle
Step 20. Inspect the table class name ("W(100%)") containing the details of each ticker symbols
Step 21. Use soup.find_all() method to detect the table class with name W(100%) and store it in variable tab
Step 22. Iterate each table row in the table class
Step 23. Iterate each table data in the table row
Step 24. With each iteration of the table data, append table data to list l1
Step 25. Append only the table data of each ticker symbol to a list proplist
Step 26. Create a list with string values Open, Previous Close, Volume and store it in variable neededprops
Step 27. Create a variable counto and initialize it to 0
Step 28. Create outer for loop to iterate each element in list proplist 
Step 29. Create a variable count and initialize it to 0
Step 30. Append ticker symbol from temp to list temp1
Step 31. Create inner for loop to Iterate each element in the sub list of proplist
Step 32. If the element is in neededprops, then append proplist[counto][count+1] to temp1
Step 33. Increment count to 1 in order to traverse through each column in the list 
Step 34. After exiting the inner loop, Append temp1 to list props
Step 35. Increment count o to 1 in order to traverse through each row in the list
Step 36. Iterate each element in props list
Step 37. Replace the (,) delimiters in p[3] with "" using replace() method
Step 38. swamp the second and third element in the prop list to display open and prev close values in order
Step 39. import csv module and open the stocks.csv file in write mode as file
Step 40. write each data element in props list as indiviual rows in stocks.csv file using writer.writerows(props)
Step 41. import sqlite3 and connect to StocksDatabase.sqlite
Step 42. Drop the table StocksTable if already exists and create a the table with the Ticker, OpenPrice, PrevClose, Volume columns 
Step 43. Open the stocks.csv file in read mode and store it invariable stockfilehandle
Step 44. Write an insert query to accept the values for the columns as a string and store it in variable insertcmd
Step 45. Iterate each stock in the stockfilehandle
Step 46. rightstrip and split each stock using , delimiter and store it in variable stockdata
Step 47. execute the sql insert command by cur.execute() method by inserting the insercmd along with stockdata[0], stockdata[1],stockdata[2],stockdata[3] as the values
Step 48. Perform commit function to store the result of the insert operation
Step 49. Execute a select operation to retrieve all the rows from StocksTable and display the output
Step 50. Exit the program

In [1]:
try:
    import urllib.request,urllib.parse,urllib.error
    import requests
    from bs4 import BeautifulSoup
    url = 'https://money.cnn.com/data/hotstocks/'
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page,'html.parser')
    table_MostActive = "wsod_dataTable wsod_dataTableBigAlt"
    data_MostActive = []
    tickersymbols = []
    temp = []
    tab = soup.find_all('table', attrs={'class':table_MostActive})[0]
    for row in tab.select('tr'):
        l1 = []
        for data in row.select('td'):
        
            l1.append(data.text)
        data_MostActive.append(l1)
    data_MostActive = data_MostActive[1:]
    #print(data_MostActive)

    for i in data_MostActive:
        tickersymbols = i[0].split(' ')[0]
        temp.append(tickersymbols)

    ticketurl = 'https://finance.yahoo.com/quote/{myticker}?p={myticker}&.tsrc=fin-srch-v1'

    proplist = []
    companylist = []

    for i in temp:
        mystockhandle = requests.get(ticketurl.format(myticker = i))
        soup = BeautifulSoup(mystockhandle.content,'html.parser')
        tab = soup.find_all('table',attrs={'class':'W(100%)'})[0]
        l1 = []
        for row in tab.select('tr'):
            for data in row.select('td'):
                l1.append(data.text)
        proplist.append(l1)
    
    neededprops = ['Open','Previous Close','Volume']
    props = []
    counto = 0

    for out in proplist:
        count = 0
        temp1 = []
    
        temp1.append(temp[counto])
        for s in out:
            if s in neededprops:
                temp1.append(proplist[counto][count+1])    
            count = count + 1
        props.append(temp1)
        counto = counto + 1
    
    for p in props:
        p[3] = p[3].replace(",","")
        t = p[1]
        p[1] = p[2]
        p[2] = t

    import csv
    with open('stocks.txt', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(props)
except:
    print("Please provide valid URL, If provided the server might be down.")


In [2]:
try:
    import sqlite3

    con = sqlite3.connect('StocksDatabase.sqlite')
    cur = con.cursor()

    cur.execute('DROP TABLE IF EXISTS StocksTable')
    cur.execute('CREATE TABLE StocksTable(Ticker TEXT, OpenPrice REAL, PrevClose REAL, Volume INTEGER)')

    stockfilehandle = open('stocks.txt','r')
    insertcmd = 'INSERT INTO StocksTable(Ticker,OpenPrice,PrevClose,Volume) VALUES (?,?,?,?)'
    for stock in stockfilehandle:
        stock = stock.rstrip()
        stockdata = stock.split(',')
        cur.execute(insertcmd,(stockdata[0],stockdata[1],stockdata[2],stockdata[3]))
        con.commit()
    
    cur.close()
    stockfilehandle.close()

    cur = con.cursor()
    print("Most Actives: ")
    cur.execute('SELECT Ticker,OpenPrice,PrevClose,Volume FROM StocksTable')
    for row in cur:
        print(row)
    cur.close()
except:
    print("Something went wrong!")

Most Actives: 
('GE', 6.67, 6.8, 120376502)
('F', 5.02, 5.09, 82660057)
('CCL', 14.9, 15.9, 63622542)
('BAC', 23.38, 24.05, 50108418)
('DAL', 24.95, 25.91, 44531739)
('NCLH', 15.25, 16.4, 43335648)
('MRO', 5.89, 6.12, 37538048)
('OXY', 16.02, 16.6, 36902354)
('T', 30.77, 30.47, 36851311)
('BA', 143.23, 141.02, 36238977)
